<hr><center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<br><strong style="color: #000000;"><em>Resize PVC volume&nbsp;</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Resize-PVC"><u>Resize PVC</u><a class="jp-InternalAnchorLink" href="#Resize-PVC" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1)&nbsp;<a href="#1" target="_self" rel="noopener">Get Storage class of PVC</a><a href="#1" target="_self" rel="noopener"><br>2)&nbsp;</a><a href="#2" target="_self" rel="noopener">Get Storage Class details</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#1" target="_self" rel="noopener">3) </a><a href="#1" target="_self" rel="noopener">Change size of PVC</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#1" target="_self" rel="noopener">4)&nbsp;</a><a href="#1" target="_self" rel="noopener">Restart the pod</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#1" target="_self" rel="noopener">5)&nbsp;</a><a href="#1" target="_self" rel="noopener">Execute a command on a Kuberentes POD</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#2" target="_self" rel="noopener">6)&nbsp;</a><a href="#2" target="_self" rel="noopener">Post Slack message</a><a href="#2" target="_self" rel="noopener"><br></a></p>
<p>&nbsp;</p>
<hr>

<h3 id="List-all-AWS-Regions">Get storage class for PVC</h3>
<p>This action fetches storage class for PVC to execute Step 2👇</p>
<blockquote>
<p>This action takes the following parameters: <code>PVCName, Namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>storageClass</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pvc {PVCName} -n {Namespace} --output=jsonpath={{.spec.storageClassName}}\\""
    }''')
task.configure(outputName="storageClass")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="List-all-AWS-Regions">Get storage class details</h3>
<p>This action fetches storage class details for PVC</p>
<blockquote>
<p>This action takes the following parameters: <code>storageClass</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>allowVolumeExpansion</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get sc {storageClass} --output=jsonpath={{.allowVolumeExpansion}}\\""
    }''')
task.configure(outputName="allowVolumeExpansion")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="List-all-AWS-Regions">Check if storage class has allowVolumeExpansion enabled</h3>
<p>This action verifies that allowVolumeExpansion is enabled for the storage class for the PVC. Assert if its not enabled.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>None</code></p>
</blockquote>

In [6]:
if allowVolumeExpansion == "" or allowVolumeExpansion is False:
    print(f'allowVolumeExpansion disabled for storage class {storageClass}, exiting')
    assert(f'allowVolumeExpansion disabled for storage class {storageClass}')
else:
    print(f'allowVolumeExpansion enabled for storage class {storageClass}')

<h3 id="Change-size-of-Kuberneted-PVC">Change size of PVC<a class="jp-InternalAnchorLink" href="#Change-size-of-Kuberneted-PVC" target="_self">&para;</a></h3>
<p>This action increases the PVC Volume by the provided Value depending upon ResizeOption chosen.</p>
<blockquote>
<p>This action takes the following parameters: <code>namespace, PVCName, value</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>None</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Optional
from unskript.enums.aws_k8s_enums import SizingOption
import pprint

from beartype import beartype
@beartype
def k8s_change_pvc_size_printer(output):
    if output is None:
        return

    pprint.pprint(output)



@beartype
def k8s_change_pvc_size(handle, namespace: str, name: str, resize_option: SizingOption, resize_value: float) -> str:
    """k8s_change_pvc_size change pvc size

        :type name: str
        :param name: Name of the PVC.

        :type resize_option: SizingOption
        :param resize_option: Option to resize the volume.

        :type resize_value: float
        :param resize_value: Based on the resize option chosen, specify the value.

        :type namespace: str
        :param namespace: Namespace of the PVC.

        :rtype: string
    """
    # Get the current size.
    kubectl_command = f'kubectl get pvc {name} -n {namespace}  -o jsonpath={{.status.capacity.storage}}'
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str(f"Error Changing PVC Size {kubectl_command}: {result.stderr}")

    currentSize = result.stdout
    currentSizeInt = int(currentSize.rstrip("Gi"))
    if resize_option == SizingOption.Add:
        newSizeInt = currentSizeInt + resize_value
    else:
        newSizeInt = currentSizeInt * resize_value
    newSize = str(newSizeInt) + "Gi"
    print(f'Current size {currentSize}, new Size {newSize}')
    kubectl_command = f'kubectl patch pvc {name} -n {namespace} -p \'{{"spec":{{"resources":{{"requests": {{"storage": "{newSize}"}}}}}}}}\''
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str(f"Error Changing PVC Size {kubectl_command}: {result.stderr}")
    print(f'PVC {name} size changed to {newSize} successfully')
    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "name": "PVCName",
    "namespace": "Namespace",
    "resize_option": "SizingOption.Add",
    "resize_value": "float(Value)"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_change_pvc_size, lego_printer=k8s_change_pvc_size_printer, hdl=hdl, args=args)

<h3 id="Get-pods-attahced-to-Kuberneted-PVC">Get pods attahced to PVC<a class="jp-InternalAnchorLink" href="#Get-pods-attahced-to-Kuberneted-PVC" target="_self">&para;</a></h3>
<p>This action gets the pods attached to a Kuberneted PVC</p>
<blockquote>
<p>This action takes the following parameters: <code>namespace, PVCName</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>podName</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_get_pods_attached_to_pvc_printer(output):
    if output is None:
        return

    print(output)



@beartype
def k8s_get_pods_attached_to_pvc(handle, namespace: str, pvc: str) -> str:
    """k8s_get_pods_attached_to_pvc get pods attached to pvc

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type namespace: str
        :param namespace: Namespace of the PVC.

        :type pvc: str
        :param pvc: Name of the PVC.

        :rtype: string
    """
    kubectl_command = f"kubectl describe pvc {pvc} -n {namespace} | awk \'/Used By/ {{print $3}}\'"
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()
    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "namespace": "Namespace",
    "pvc": "PVCName"
    }''')
task.configure(outputName="podName")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_get_pods_attached_to_pvc, lego_printer=k8s_get_pods_attached_to_pvc_printer, hdl=hdl, args=args)

<p>This action simply extracts the&nbsp; pod name attached to a Kuberneted PVC</p>

In [ ]:
podName = podName.strip()
print(f'Pod {podName} attached to PVC {PVCName}')

<h3 id="List-all-AWS-Regions">Restart the pod</h3>
<p>This action restarts the pod. If RestartPodsAfterResize is enabled, it restarts the pods attached to the PVC.</p>
<p>NOTE: This is not required if the kubernetes has <span style="color: rgb(53, 152, 219);">ExpandInUsePersistentVolumes</span> enabled.</p>
<blockquote>
<p>This action takes the following parameters: <code>namespace, podName</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>None</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl delete pod {podName} -n {Namespace}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "RestartPodsAfterResize==True",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="List-all-AWS-Regions">Execute a command on a Kuberentes POD</h3>
<p>This action verifies resize by running 'df-kh' on the pod attached to the PVC.</p>
<blockquote>
<p>This action takes the following parameters: <code>namespace, podName</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>None</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from posixpath import split
from typing import List
import pprint
from kubernetes import client
from kubernetes.stream import stream
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_exec_command_on_pod_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def k8s_exec_command_on_pod(handle, namespace: str, podname: str, command: str) -> str:
    """k8s_exec_command_on_pod executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type namespace: str
        :param namespace: Kubernetes namespace.

        :type podname: str
        :param podname: Kubernetes Pod Name.

        :type command: str
        :param command: Commands to execute on the Pod.

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    coreApiClient = client.CoreV1Api(api_client=handle)

    try:
        resp = stream(coreApiClient.connect_get_namespaced_pod_exec,
                      podname,
                      namespace,
                      command=command.split(),
                      stderr=True,
                      stdin=True,
                      stdout=True,
                      tty=False
                      )
    except Exception as e:
        resp = f'An Exception occured while executing the command {e}'

    return resp


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "[\\"df\\", \\"-kh\\"]",
    "namespace": "Namespace",
    "podname": "podName"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_exec_command_on_pod, lego_printer=k8s_exec_command_on_pod_printer, hdl=hdl, args=args)

<h3 id="List-all-AWS-Regions">Run kubectl commands on local k8s cluster</h3>
<p>This action further verifies the resize by running commands on the local k8s cluster and gets the new size.</p>
<blockquote>
<p>This action takes the following parameters: <code>namespace, PVCName</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>newSize</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pvc {PVCName} -n {Namespace}  -o jsonpath={{.status.capacity.storage}}\\""
    }''')
task.configure(outputName="newSize")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="List-all-AWS-Regions">Post Slack Message</h3>
<p>This action posts a slack message notifying the new size of the PVC.</p>
<blockquote>
<p>This action takes the following parameters: <code>Channel</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>None</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def slack_post_message_printer(data):
    if data != None:
        pprint.pprint(data)

@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.reponse['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.respones['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

        return f"Unable to send message on {channel}"
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "channel": "Channel",
    "message": "f\\"PVC {PVCName} successfully resized to {newSize}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "if len(Channel)!=0",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;&para;">Conclusion</h3>
<p>In this Runbook, we were able successfully resize a PVC using unSkript's K8s actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>